#Download the Dataset


In [ ]:
# First Download the dataset to the same directory 

#Preprocessing of the dataset

In [ ]:
import glob
import numpy as npn
from osgeo import gdal
import tensorflow as tf
import numpy as np


def bandstoarray(ds):
    band1 = ds.GetRasterBand(1) # Red channel 
    band2 = ds.GetRasterBand(2) # Green channel 
    band3 = ds.GetRasterBand(3) # Blue channel


    b1 = band1.ReadAsArray() 
    b2 = band2.ReadAsArray() 
    b3 = band3.ReadAsArray() 


    rgb = np.dstack((b1,b2,b3))


    rgb_pixel=tf.reshape(rgb, (rgb.shape[0]*rgb.shape[1], 3))

    return rgb_pixel









def Create_dataset(path_file_rgb,glob_file_rgb,path_file_B567,glob_file_B567):
    n=0

    
    nfiles=len(sorted(glob.glob(glob_file_rgb)))
    print("Number of files",nfiles)

    for num in range(nfiles):  # range(nfiles)
        
        path_name_rgb=path_file_rgb.format(num)
        ds_rgb = gdal.Open(path_name_rgb)

        path_name_B567=path_file_B567.format(num)
        ds_B567 = gdal.Open(path_name_B567)


        if (ds_rgb.RasterCount==3):
            

            RGB=bandstoarray(ds_rgb)

            B567=bandstoarray(ds_B567)


            Total=np.concatenate([RGB, B567], axis=1)

            np.random.shuffle(Total)
            np.random.shuffle(Total)
            np.random.shuffle(Total)
  


            n_of_rows = Total.shape[0]

            n_indices = np.random.choice(n_of_rows, size=50, replace=False)#50 samples

            random_rows = Total[n_indices, :]


            if (n == 0):
                
                B_T=random_rows
                


            else:


                Prev= B_T

                B_T=np.concatenate([Prev,random_rows], axis=0)
            
            n=n+1

            output.clear()
            print("Files completed ",n)



         


   
    print("Parameters rgb_pixel",B_T.shape)
    

    return n ,B_T
    
  


In [ ]:
path_file_1="/content/Atmo_corr/Train/RGB/RGB_{}.tif" 
glob_file_1="/content/Atmo_corr/Train/RGB/RGB_*.tif" 

path_file_2="/content/Atmo_corr/Train/B567/TAR_{}.tif" 
glob_file_2="/content/Atmo_corr/Train/B567/TAR_*.tif" 

n1,All_train=Create_dataset(path_file_1,glob_file_1,path_file_2,glob_file_2)
print('number of training  samples: ', All_train.shape)


# shuffle for several times : dataset is so large 
np.random.shuffle(All_train)
np.random.shuffle(All_train)
np.random.shuffle(All_train)
np.random.shuffle(All_train)
np.random.shuffle(All_train)


X=All_train[:,[0,1,2]]
Y=All_train[:,[3,4,5]]


print(X.shape,Y.shape)




Files completed  18370
Parameters rgb_pixel (918500, 6)
number of training  samples:  (918500, 6)
(918500, 3) (918500, 3)


In [ ]:
# Prepare it for deep learning model 

X=np.reshape(X, (X.shape[0],1,X.shape[1]))

Y=np.reshape(Y, (Y.shape[0],1,Y.shape[1]))

print(X.shape,Y.shape)

(918500, 1, 3) (918500, 1, 3)


#The proposed model


In [1]:
from tensorflow.python.keras.layers import  Lambda,Conv1D, Activation
from tensorflow.python import keras
import tensorflow as tf



def Conv_block(x_in, IN,filters):
    
    x1 = Conv1D(filters, 3,activation='relu',padding='same')(x_in)        

    x3 = Conv1D(filters, 3,padding='same')(x1) 


    X_T = tf.keras.layers.concatenate([x3,IN])

    return X_T







def Variant_model():

    inputs = keras.Input(shape=(1,3))  

    Y0=Conv_block(inputs,inputs,8)
    Y1=Conv_block(Y0,inputs,16)
    Y2=Conv_block(Y1,inputs,32)
    Y3=Conv_block(Y2,inputs,64)
    Y4=Conv_block(Y3,inputs,64)
    Y5=Conv_block(Y4,inputs,128)
   

    Out = Conv1D(3, 3,padding='same')(Y5)
    


    return keras.Model(inputs=inputs, outputs=Out, name="COM")


COM=Variant_model()

COM.summary()

Model: "COM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 3)]       0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1, 8)         80          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1, 8)         200         conv1d[0][0]                     
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 1, 11)        0           conv1d_1[0][0]                   
                                                                 input_1[0][0]                  

#Training process


In [ ]:
from keras import optimizers


checkpoint_filepath = '/content/checkpoint'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


Adam = optimizers.Adam(lr=0.001)  
COM.compile(optimizer=Adam, loss='mse')

history1=COM.fit(X,Y,validation_split=0.2,batch_size=256,callbacks=[model_checkpoint_callback],epochs=100)

In [ ]:
#Save model 
COM.save("my_model")

#Apply for test

In [ ]:
from tensorflow import keras
#!unzip my_model.zip     

reconstructed_model = keras.models.load_model("my_model")

In [ ]:
#install some libraries 
!pip install hdf5storage
!pip install image-similarity-measures
import image_similarity_measures
from image_similarity_measures.quality_metrics import  psnr,sam,ssim

##Preprocess Test Dataset

In [ ]:
import torch
import torch.nn as nn
import hdf5storage
def Test_bandstoarray(ds):
    band1 = ds.GetRasterBand(1) # Red channel 
    band2 = ds.GetRasterBand(2) # Green channel 
    band3 = ds.GetRasterBand(3) # Blue channel


    b1 = band1.ReadAsArray()  
    b2 = band2.ReadAsArray() 
    b3 = band3.ReadAsArray() 

           
    rgb = np.dstack((b1,b2,b3))


    return rgb






def batch_RMSE(im_true, im_fake, data_range=255.):

    im_true=torch.from_numpy(np.array(im_true))
    im_fake=torch.from_numpy(np.array(im_fake))

    #print("Tensor size ",im_true.size())

    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).mul_(data_range).resize_(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).mul_(data_range).resize_(N, C*H*W)
    mse = nn.MSELoss(reduce=False)
    err = mse(Itrue, Ifake).sqrt_().sum(dim=1, keepdim=True).div_(C*H*W)
    return torch.mean(err)


def batch_rRMSE(im_true, im_fake, data_range=255.):
    im_true=torch.from_numpy(np.array(im_true))
    im_fake=torch.from_numpy(np.array(im_fake))

    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).mul_(data_range).resize_(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).mul_(data_range).resize_(N, C*H*W)
    mse = nn.MSELoss(reduce=False)
    err = mse(Itrue, Ifake).sqrt_().div_(Itrue).sum(dim=1, keepdim=True).div_(C*H*W)
    return torch.mean(err)


def reformulate(X):
     PR = np.expand_dims(X, axis=0)
     #print(PR.shape)
     PR=  np.transpose(PR, [0,3,1,2])
     #print(PR.shape)
  
     return PR





def apply_test(path_file_rgb,glob_file_rgb,path_file_B567,glob_file_B567,reconstructed_model):
    n=0
    RMSE=0
    rRMSE=0
    SAM=0
    PSNR=0
    SSIM=0

    
    nfiles=len(sorted(glob.glob(glob_file_rgb)))
    print("Number of files",nfiles)
    mycount=0

    ND=20  # Change the values   nfiles

    SM = np.zeros(ND)
    SM1 = np.zeros(ND)
    ps1 = np.zeros(ND)
    ps2 = np.zeros(ND)
    for num in range(ND):  # range(nfiles)
        print(num)
        mycount=mycount+1
        path_name_rgb=path_file_rgb.format(num)
        ds_rgb = gdal.Open(path_name_rgb)

        path_name_B567=path_file_B567.format(num)
        ds_B567 = gdal.Open(path_name_B567)

        RGB=Test_bandstoarray(ds_rgb)

        B567=Test_bandstoarray(ds_B567)

        
        # Predict   
        
        RGB_predict=tf.reshape(RGB, (RGB.shape[0]*RGB.shape[1],1, 3))

        B567_predictd=reconstructed_model.predict(RGB_predict)

        B567_predictd=np.reshape(B567_predictd, (RGB.shape[0],RGB.shape[1], 3))  # tf 

        B567_predictd=np.clip(B567_predictd, 0, 1)
        B567=np.clip(B567, 0, 1)

        psnr2 = tf.image.psnr(B567_predictd, B567, max_val=1.0)
        #print(psnr2)
        SM1[num]=sam(B567,B567_predictd)
        ps1[num]=psnr(B567,B567_predictd,max_p=1 )
        ps2[num]=psnr2
        # add other metrics 
        PR=reformulate(B567_predictd)
        OR=reformulate(B567)

        RMSE = RMSE+batch_RMSE(OR, PR)


        rRMSE = rRMSE+batch_rRMSE(OR, PR)

        SAM = SAM+sam(B567,B567_predictd)

        PSNR=PSNR+psnr(B567,B567_predictd,max_p=1)

        SSIM=SSIM+ssim(B567,B567_predictd,max_p=1)



    return RMSE/mycount,rRMSE/mycount , SAM/mycount,PSNR/mycount,SSIM/mycount

In [ ]:

path_file_1="/content/Atmo_corr/Test/RGB/RGB_{}.tif" 
glob_file_1="/content/Atmo_corr/Test/RGB/RGB_*.tif" 

path_file_2="/content/Atmo_corr/Test/B567/TAR_{}.tif" 
glob_file_2="/content/Atmo_corr/Test/B567/TAR_*.tif" 

n1,n2,n3,n4,n5=apply_test(path_file_1,glob_file_1,path_file_2,glob_file_2,reconstructed_model)

print("RMSE",n1,"rRMSE",n2,"SAM",n3,"PSNR",n4,"SSIM",n5)


Number of files 5207
0


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
RMSE tensor(5.2431) rRMSE tensor(0.0845) SAM 2.88199103474617 PSNR 31.969075202941895 SSIM 0.8941062025041651
